# HHL Algorithm
HHL Algorithm to solve system of linear equations:
$$Ax = b$$
We represent x and b through qubits
$$\ket{x} \quad \text{and} \quad \ket{b}$$

In [163]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np

# Number of qubits and classical bits.
# ----------------------------------------
num_ancilla = 1

# Qubits for estimating the eigenvalues.
# Increase this number to get a more accurate estimation of the eigenvalues.
num_estimation_qubits = 2

# Qubits to hold the state of b.
num_bqubits = 1

num_classical = num_estimation_qubits
# ----------------------------------------

# Create the quantum- and classical registers.
ar = QuantumRegister(num_ancilla, "ancilla")
qr = QuantumRegister(num_estimation_qubits, "estimation_qubit")
br = QuantumRegister(num_bqubits, "b_qubit")
cr = ClassicalRegister(num_classical, "cl_bit")

# Create the quantum circuit.
hhl_qc = QuantumCircuit(ar, qr, br, cr)

## 1. Phase Estimation.
Implement the phase estimation.

### 1.1 Apply the Hadamard Gate to the "estimation qubits".

In [164]:
hhl_qc.h(qr)

hhl_qc.draw()

ancilla: ─────
                    ┌───┐
estimation_qubit_0: ┤ H ├
                    ├───┤
estimation_qubit_1: ┤ H ├
                    └───┘
           b_qubit: ─────
                         
          cl_bit: 2/═════

### 1.2 Prepare the state of |b>
Prepare the state of
$$\ket{b} = \begin{pmatrix} 1 \\ 0\end{pmatrix},  \quad  \ket{b} = \begin{pmatrix} 0 \\ 1\end{pmatrix}$$
by setting num_eigenvalue = 0 or 1 respectively.

In [165]:
# Set num_eigenvalue = 0 or 1 to get |b> = [1, 0] = |0> or |b> = [0, 1] = |1> respectively.
num_eigenvalue = 0

if num_eigenvalue == 0:
    # Expected decimal value for phase estimation:
    expected_value = 1 / np.pi
elif num_eigenvalue == 1:
    # Expected decimal value for phase estimation:
    expected_value = 1 / (2*np.pi)
    hhl_qc.x(br)

### 1.3 Estimate the matrix exponent of A
To keep it simple, we choose
$$A = \begin{pmatrix} 2 & 0 \\ 0 & 1\end{pmatrix}$$
This enables us to easily calculate
$$e^{iA} = \begin{pmatrix} e^{2i} & 0 \\ 0 & e^{i}\end{pmatrix}$$
because A is a diagonal matrix.
It also enables us to easily create a quantum gate for the matrix exponent:
$$U(0,1,0) = \begin{pmatrix} 1 & 0 \\ 0 & e^{i}\end{pmatrix}, \quad
X = \begin{pmatrix} 0 & 1 \\ 1 & 0\end{pmatrix}, \quad
U(0,1,1) = \begin{pmatrix} 1 & 0 \\ 0 & e^{2i}\end{pmatrix}
$$
where in general
$$U(\theta, \phi, \lambda) = \begin{pmatrix} \cos\left( \frac{\theta}{2}\right) & -e^{i\lambda} \sin\left( \frac{\theta}{2}\right)\\ 
e^{i\theta} \sin\left( \frac{\theta}{2}\right) & e^{i(\phi + \lambda)}\cos\left( \frac{\theta}{2}\right)\end{pmatrix} $$
and it follows that
$$e^{iA} = U(0,1,0) \cdot X \cdot U(0,1,1) \cdot X$$

In [166]:
# Create e^(iA) as a custom gate.
eia = QuantumCircuit(1)
eia.u(0,1,0,0)
eia.x(0)
eia.u(0,1,1,0)
eia.x(0)

eia_gate = eia.to_gate()
ctrl_eia_gate= eia_gate.control(1)

eia.draw()

┌──────────┐┌───┐┌──────────┐┌───┐
q: ┤ U(0,1,0) ├┤ X ├┤ U(0,1,1) ├┤ X ├
   └──────────┘└───┘└──────────┘└───┘

In [167]:
# Check if the operator eia is what we intended.
import qiskit.quantum_info as qi
op = qi.Operator(eia)
eia_array = op.data
expected_array = np.array([[np.exp(2j), 0],
                          [0, np.exp(1j)]])

# Difference between eia_array and expected array. Should be 0. 
print(eia_array - expected_array)

[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


Append ctrl_eia_gates to circuit, according to phase estimation algorithm.

In [168]:
# Append ctrl_eia_gates to circuit.
for i in range(0, num_estimation_qubits):
    for j in range(0, 2**i):
        hhl_qc.append(ctrl_eia_gate, [qr[i], br[0]])

# Draw circuit if needed.
# hhl_qc.decompose().draw()

### 1.4 Create the Inverse Quantum Fourier Transformation Gate (IQFT).
Create the IQFT as a custom gate from scratch.

In [169]:
# Create the IQFT as a custom gate.
iqft = QuantumCircuit(qr)

# Implement the swap gates.
for i in range(0, num_estimation_qubits // 2):
    iqft.swap(i, num_estimation_qubits - i - 1)
    
# Implement the Hadadmard Gates and controlled P-Gates.
for i in range(0, num_estimation_qubits):
    for j in range(2, 2 + i):
        iqft.cp(theta=-2*np.pi / 2**(3 + i - j), control_qubit=j-2, target_qubit=i)
    iqft.h(i)

# Create a custom gate from circuit.
iqft_gate = iqft.to_gate()

# Draw circuit.
# iqft.draw()

In [170]:
# Append the IQFT to the circuit.
hhl_qc.append(iqft, qr)

# Draw circuit if needed.
# hhl_qc.decompose().draw()

### 1.5 Testing the phase estimation.
We will test the phase estimation by approximating the eigenvalues of e^(ia)
$$e^{2i} \quad \text{and} \quad e^{i}$$
which belong to the eigenvectors
$$\ket{b} = \begin{pmatrix} 1 \\ 0\end{pmatrix},  \quad  \ket{b} = \begin{pmatrix} 0 \\ 1\end{pmatrix}$$
respectively.

In [171]:
# Copy hhl_qc circuit so we can test the phase estimation with it.
pe_qc = hhl_qc.copy()

# Measure all estimation qubits.
pe_qc.measure(qr, cr)

# Draw circuit if needed.
# pe_qc.decompose().draw()

In [172]:
from qiskit import transpile
from qiskit_aer import Aer

shots = 2000
backend = Aer.get_backend("qasm_simulator")
tqc = transpile(pe_qc, backend)
job = backend.run(tqc, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)

{'11': 74, '00': 112, '10': 239, '01': 1575}


### 1.6 Post-Processing
If the eigenvalue for some eigenvector |b> is
$$e^{i\theta}$$
and the estimation_qubits are
$$q_{m-1}, q_{m-2}, ... , q_{0}$$
with m = num_estimation_qubits, our variable from earlier, then according to the phase estimation we get the approximation
$$\frac{\theta}{2\pi} \approx q_{m-1}2^{-1} + q_{m-2}2^{-2} + ... + q_{0}2^{-m}$$
We have set the circuit measurement up in such a way that
$$q_k = estimation\_qubit\_k \quad k \in \lbrace 0,...,m-1\rbrace$$
and so that a "count result" returned by qiskit is returned in the form
$$result = \ket{q_{m-1} q_{m-2} ... q_{0}}$$
The 2 expected values from our estimation, depending on the choice of |b> earlier,  are therefore
$$\begin{align}
\theta &= 2 \Rightarrow \frac{\theta}{2\pi} = \frac{1}{\pi} \approx 0.31830, \quad for \quad \ket{b} = \begin{pmatrix} 1 \\ 0\end{pmatrix} \quad \text{and} \\
\theta &= 1 \Rightarrow \frac{\theta}{2\pi} = \frac{1}{2\pi} \approx 1.59154, \quad for \quad \ket{b} = \begin{pmatrix} 0 \\ 1\end{pmatrix}
\end{align}
$$

In [173]:
# Order results by probability/count.
result_lst = list(counts.items())
result_lst.sort(key=lambda tup: tup[1], reverse=True)
decimal_lst = []

# Stop printing after stop_print results.
stop_print=10

# Check what was estimated. Only print the 10 most measured results.
print(f"{"Probability":10s} |  {"Binary":12s} | {"Decimal":9s} | Approximated Eigenvalue")
for binary, count in result_lst:
    i = -1
    decimal = 0
    for digit in binary:
        decimal += int(digit) * 2**(i)
        i -= 1
    decimal_lst.append(decimal)
    print("-------------------------------------------------------------")
    print(f"{count / shots:0.8f}  | 0.{binary} | {decimal:0.7f}  | {decimal * 2 * np.pi:0.7f}")
    stop_print -= 1
    if stop_print == 0:
        break

Probability |  Binary       | Decimal   | Approximated Eigenvalue
-------------------------------------------------------------
0.78750000  | 0.01 | 0.2500000  | 1.5707963
-------------------------------------------------------------
0.11950000  | 0.10 | 0.5000000  | 3.1415927
-------------------------------------------------------------
0.05600000  | 0.00 | 0.0000000  | 0.0000000
-------------------------------------------------------------
0.03700000  | 0.11 | 0.7500000  | 4.7123890


In [174]:
# Most frequent result vs. expected value.
# Increase the number of estimation qubits, to get more accurate approximations of the eigenvalues.
print("Phase Estimation: Most frequent value - Expected value")
print(decimal_lst[0] - expected_value)

Phase Estimation: Most frequent value - Expected value
-0.06830988618379069


At 10 qubits we get a consistent error between the approximated and the actual eigenvalue < 10^(-4) .
Now we will implement the rest of the algorithm.

## 2. Controlled Rotation of the ancillary qubit.

We would like to put the whole system in the following state
$$\sum_{j=1}^{n} \beta_j \left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\ket{0}_a + \frac{C}{\lambda_j}\ket{1}_a\right) \ket{\lambda_j}_r \ket{u_j}_b$$
Where
$$
\begin{align}
&\ket{\cdot}_a, \ket{\cdot}_r, \ket{\cdot}_b \quad \text{stands for the ancillary qubit, estimation qubits and b vector qubits respectively.} \\
&j \quad \text{iterates over the n eigenvalues. In our case n = 2.} \\
&C \quad \text{is just a global phase such that} \quad C \leq \min_{j \in \lbrace 1, ..., n\rbrace}{\lambda_j} \\
&\lambda_j, u_j \quad \text{are the corresponding eigenvalues and eigenvectors of} \quad A \\
&\text{Eventually we have} \quad \ket{b} = \sum_{j = 1}^n \beta_j \ket{u_j}, \quad \text{The eigenvector representation of} \quad \ket{b}
\end{align}
$$
Currently our system is still in the state
$$\sum_{j=1}^{n} \beta_j \ket{0}_a  \ket{\lambda_j}_r \ket{u_j}_b$$
To transform it we will use that the normalized eigenvectors of A are 
$$\begin{pmatrix} 1 \\ 0\end{pmatrix},  \quad  \begin{pmatrix} 0 \\ 1\end{pmatrix}$$
which are exactly the two choices for |b> we made earlier. Technically we can not always use this trick, since |b> is not always equal to an eigenvector, but to avoid complications in this toy example, we will use it. Normally, we would use the eigenvalues calculated through our phase estimation but it would complicate matters too much for now. This let's us transform the state of the ancillary qubit controlled by the eigenvector. In our case we can choose C = 1, since the smallest eigenvalue of A is 1. This yields the following cases
$$
\ket{0}_a \to 
\begin{cases}
\frac{\sqrt{3}}{2}\ket{0}_a + \frac{1}{2}\ket{1}_a, \quad &\text{if } u = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \\
\ket{1}_a, \quad &\text{if } u = \begin{pmatrix} 0 \\ 1 \end{pmatrix}
\end{cases}
$$
We can implement these cases via controlled U rotations through the angles
$$\lvert \arccos\left( \frac{1}{2}\right)\rvert = \frac{\pi}{3}, \lvert\arccos\left(1\right)\rvert = \pi $$
Let's add those rotations to our circuit.

In [175]:
# Add the controlled rotations to the circuit.

# Controlled rotation for eigenvalue lambda = 1.
hhl_qc.cu(np.pi, 0, 0, 0, control_qubit=br[0], target_qubit=ar[0])

# Inverse control for second eigenvalue lambda = 2 (Apply gate when in state |0> not in state |1>).
hhl_qc.x(br[0])

# Controlled rotation for eigenvalue lambda = 2.
hhl_qc.cu(np.pi / 3, 0, 0, 0, control_qubit=br[0], target_qubit=ar[0])

# Rotate b back to original state.
hhl_qc.x(br[0])

In [176]:
# hhl_qc.draw()

## 3. Reverse the IQFT with the QFT.
Reset the estimation qubits back to their original states (|0>) by reversing the IQFT from the phase estimation.
The inverse of the IQFT is the QFT.

### 3.1 Implementing the QFT as a custom gate.

In [177]:
# Create the QFT as a custom gate.
qft = QuantumCircuit(qr)
    
# Implement the Hadadmard Gates and controlled P-Gates.
for i in range(0, num_estimation_qubits):
    qft.h(num_estimation_qubits - i - 1)
    for j in range(0, num_estimation_qubits - i - 1):
        qft.cp(theta=2*np.pi / 2**(j + 2), control_qubit=j, target_qubit=num_estimation_qubits - i - 1)

# Implement the swap gates.
for i in range(0, num_estimation_qubits // 2):
    qft.swap(i, num_estimation_qubits - i - 1)
    
# Create a custom gate from circuit.
qft_gate = qft.to_gate()

# Draw circuit.
qft.draw()

┌───┐   
estimation_qubit_0: ──────■───────┤ H ├─X─
                    ┌───┐ │P(π/2) └───┘ │ 
estimation_qubit_1: ┤ H ├─■─────────────X─
                    └───┘

### 3.2 Append the QFT to the estimation register and add a measurement on the b qubit and the ancillary qubit.

In [178]:
# Add the QFT to the hhl_qc circuit.
hhl_qc.append(qft, qr)

# Add a measurement to the |b> qubit which contains x now.
hhl_qc.measure([br[0], ar[0]], cr[0:2])

In [179]:
# hhl_qc.decompose().draw()

## 4. Run the circuit and look at the results.
We will calculate the solution |x> by measuring the frequency of the ancillary states.
We don't really need to measure |b> since we know that |b> is an eigenvector of A, but in general we would need to do it.

In [188]:
shots = 10000
backend = Aer.get_backend("qasm_simulator")
hhl_qc_t = transpile(hhl_qc, backend)
job = backend.run(hhl_qc_t, shots=shots)
print(job)
result = job.result()
counts = result.get_counts()
print(counts)

{'10': 2498, '00': 7502}


In [42]:
hhl_qc.draw()

»
           ancilla: ──────────────────────────────────────────────────»
                    ┌───┐                                             »
estimation_qubit_0: ┤ H ├───────■─────────────────────────────────────»
                    ├───┤       │                                     »
estimation_qubit_1: ┤ H ├───────┼──────────────■──────────────■───────»
                    ├───┤       │              │              │       »
estimation_qubit_2: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_3: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_4: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_5: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_6: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_7: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_8: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    ├───┤       │              │              │       »
estimation_qubit_9: ┤ H ├───────┼──────────────┼──────────────┼───────»
                    └───┘┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐»
           b_qubit: ─────┤ circuit-198 ├┤ circuit-198 ├┤ circuit-198 ├»
                         └─────────────┘└─────────────┘└─────────────┘»
         cl_bit: 10/══════════════════════════════════════════════════»
                                                                      »
«                                                                 »
«           ancilla: ─────────────────────────────────────────────»
«                                                                 »
«estimation_qubit_0: ─────────────────────────────────────────────»
«                                                                 »
«estimation_qubit_1: ─────────────────────────────────────────────»
«                                                                 »
«estimation_qubit_2: ───────■──────────────■──────────────■───────»
«                           │              │              │       »
«estimation_qubit_3: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_4: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_5: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_6: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_7: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_8: ───────┼──────────────┼──────────────┼───────»
«                           │              │              │       »
«estimation_qubit_9: ───────┼──────────────┼──────────────┼───────»
«                    ┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐»
«           b_qubit: ┤ circuit-198 ├┤ circuit-198 ├┤ circuit-198 ├»
«                    └─────────────┘└─────────────┘└─────────────┘»
«         cl_bit: 10/═════════════════════════════════════════════»
«                                                                 »
«                                                                 »
«           ancilla: ─────────────────────────────────────────────»
«                                                                 »
«estimation_qubit_0: ─────────────────────────────────────────────»
«                 

## 5. Postprocessing.
Postprocess the results to get our vector x.
We will calculate
$$
\sqrt{\frac{\#count_j}{\#shots}} \approx \frac{\beta_j}{\lambda_j}
$$
by counting the measurements where the ancilla qubit is 1 (the first bit in the resulting bit string from the left).
We will match the corresponding eigenvector by reading out the second bit (b-vector bit) in the bit string.

In [196]:
from math import sqrt

# The first bit in our results represents the |b> qubit.
# The second qubit represents the ancilla qubit.
for bit_string in counts.keys():
    if bit_string.startswith("1"):
        beta_lambda = sqrt(counts[bit_string] / shots)
        if bit_string[1] == "0":
            x = np.array([beta_lambda, 0])
            b = np.array([1, 0])
        else:
            x = np.array([0, beta_lambda])
            b = np.array([0, 1])
            
# Matrix A
A = np.array([[2,0],[0,1]])

# Solution of the algorithm.
print(f"Solution vector x:\nx = {x}")
print()

# Calculate difference of solution and b.
# The difference should decrease by increasing the number of shots,
# hecause it gives a better approximation of the phase.
print(f"Difference between b calculated by the solution and the actual b:\nA*x - b = {np.matmul(A,x) - b}")

Solution vector x:
x = [0.49979996 0.        ]

Difference between b calculated by the solution and the actual b:
A*x - b = [-0.00040008  0.        ]


## Remarks

We took some shortcuts along the way. Here they are listed:
1. We only employed the algorithm for a diagonal matrix A. We did this because of the hamiltonian/quantum simulation, which would have been way more complicated.
2. We did not base the rotation of the ancilla qubit on the results of the phase estimation, because it would have taken significantly more time to implement the rotations based on our estimation bits (the general approach).
3. We kept it simple by only choosing b = [1,0], [0,1], to simplify the post-procesesing, although it's easily possible to adapt the post processing to a general choice of b.